# Create files mapping FACS well and plate to basic allele metadata

This script takes cherry picking and allele info files as input and produces a table relating FACS well and plate IDs to allele IDs and symbols. 

In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
dualipa_inputs = "../1_inputs"
dualipa_outputs = "../3_outputs"
allele_collection_inputs = "../../../1_allele_collection/1_inputs"
allele_collection_outputs = "../../../1_allele_collection/3_outputs"

In [14]:
# Get allele metadata
allele_info_df = pd.read_table(f"{allele_collection_inputs}/VarChamp_Consolidated_allele_collection.tsv")
other_cols = [i for i in allele_info_df.columns if "mutation_id_" not in i]
mut_cols = [i for i in allele_info_df.columns if "mutation_id_" in i]

allele_info_df = allele_info_df.melt(id_vars=other_cols, 
                                     value_vars=mut_cols, 
                                     var_name='mut_type', 
                                     value_name='mut_id')
allele_info_df

,orf_id,symbol,nt_change,aa_change,ensembl_gene_id,chr,chr_pos_38,spdi,ccsb_allele_id,ccsb_mutation_id,entry_plate,entry_well,collection,sequence,mut_type,mut_id
0,2,GBA,160G>C,Val54Leu,ENSG00000177628,1,155240033,NC_000001.11:155240033:C:G,ALE0000584,CCSBVarC000001,GDEhDisVCh_40054,F12,RC4,ATGGAGTTTTCAAGTCCTTCCAGAGAGGAATGTCCCAAGCCTTTGA...,mutation_id_rc4,6.0
1,2,GBA,670C>T,Leu224Phe,ENSG00000177628,1,155238225,NC_000001.11:155238225:G:A,ALE00000002,CCSBVarC000002,GDEhDisVCh_40054,C12,RC4,ATGGAGTTTTCAAGTCCTTCCAGAGAGGAATGTCCCAAGCCTTTGA...,mutation_id_rc4,73.0
2,2,GBA,887G>A,Arg296Gln,ENSG00000177628,1,155237453,NC_000001.11:155237453:C:T,ALE00000003,CCSBVarC000003,GDEhDisVCh_40054,D12,RC4,ATGGAGTTTTCAAGTCCTTCCAGAGAGGAATGTCCCAAGCCTTTGA...,mutation_id_rc4,113.0
3,2,GBA,1448T>C,Leu483Pro,ENSG00000177628,1,155235252,NC_000001.11:155235252:A:G,ALE00000004,CCSBVarC000004,GDEhDisVCh_40054,E12,RC4,ATGGAGTTTTCAAGTCCTTCCAGAGAGGAATGTCCCAAGCCTTTGA...,mutation_id_rc4,231.0
4,25,SLC9A3R1,328C>G,Leu110Val,ENSG00000109062,17,74749174,NC_000017.11:74749174:C:G,ALE00000005,CCSBVarC000005,GDEhDisVCh_40045,E04,RC4,ATGAGCGCGGACGCAGCGGCCGGGGCGCCCCTGCCCCGGCTCTGCT...,mutation_id_rc4,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25459,8999,TTR,355G>A,Asp119Asn,ENSG00000118271,18,31598586,NC_000018.10:31598586:G:A,ALE00008468,CCSBVarC008481,NaN,NaN,Edgotyping3,ATGGCTTCTCATCGTCTGCTCCTCCTCTGCCTTGCTGGACTGGTAT...,mutation_id_edgotyping3,390.0
25460,7323,PMP22,152A>G,His51Arg,ENSG00000109099,17,15259120,NC_000017.11:15259120:T:C,ALE00008469,CCSBVarC008466,NaN,NaN,Edgotyping3,ATGCTCCTCCTGTTGCTGAGTATCATCGTCCTCCACGTCGCGGTGC...,mutation_id_edgotyping3,394.0
25461,2651,CLN6,89G>C,Gly30Ala,ENSG00000128973,15,68218645,NC_000015.10:68218645:C:G,ALE00008470,CCSBVarC008436,NaN,NaN,Edgotyping3,ATGGAGGCGACGCGGAGGCGGCAGCACCTGGGAGCGACGGGCGGCC...,mutation_id_edgotyping3,436.0
25462,7323,PMP22,403G>A,Ala135Thr,ENSG00000109099,17,15230997,NC_000017.11:15230997:C:T,ALE00008471,CCSBVarC008467,NaN,NaN,Edgotyping3,ATGCTCCTCCTGTTGCTGAGTATCATCGTCCTCCACGTCGCGGTGC...,mutation_id_edgotyping3,1840.0


In [15]:
# Get gene-level metadata
gene_info_df = pd.read_csv(f"{allele_collection_outputs}/orf_info.csv")
gene_info_df = gene_info_df[['orf_id', 'symbol']]
orf_to_symbol = gene_info_df.set_index("orf_id")["symbol"].to_dict()
orf_to_symbol = {str(k): v for k, v in orf_to_symbol.items()}

Vidal lab ran experiment once and analyzed the data. Some alleles didn't pass the qa/qc thresholds or got no data. Batch5 is is a re-run for all alleles that failed in the first set of batches. Here, we process everything together from scratch. An allele should always be compared to the wt on the same batch and plate. The summary stats can then be compared across batches. 

In [16]:
# Read in cherry picking files for both the initial batches and the second round
cp_col = ['orf_id', 'mut_id', 'node_type','dest_pla_id', 'dest_pla', 'dest_well']

pdest_layout_initial_batches_df = pd.read_csv(f'{dualipa_inputs}/cherry_picking_files/summary_CP_VUSDUALXP.csv', index_col=0)[cp_col]
pdest_layout_b5_df = pd.read_csv(f'{dualipa_inputs}/cherry_picking_files/summary_CP_VUSDUALXP_batch5.csv', index_col=0)[cp_col]

pdest_layout_df = pd.concat([pdest_layout_b5_df, pdest_layout_initial_batches_df])
pdest_layout_df
### This doesn't have well A01

,orf_id,mut_id,node_type,dest_pla_id,dest_pla,dest_well
78,56360.0,0.0,wt,23,VUSDUALXP_23,A12
79,56360.0,0.0,wt,23,VUSDUALXP_23,B12
80,56360.0,0.0,wt,23,VUSDUALXP_23,C12
81,56360.0,0.0,wt,23,VUSDUALXP_23,D12
10,9905.0,0.0,wt,23,VUSDUALXP_23,F02
...,...,...,...,...,...,...
119,NaN,NaN,pdest_empty,20,VUSDUALXP_20,A08
124,NaN,NaN,pdest_empty,21,VUSDUALXP_21,A07
125,NaN,NaN,pdest_empty,21,VUSDUALXP_21,A08
130,NaN,NaN,pdest_empty,22,VUSDUALXP_22,A07


In [17]:
seq_confirm_code_df = pd.read_table(f"{allele_collection_inputs}/sequence_confirmation_class_code.tsv")
with pd.option_context('display.max_columns', None):
    display(seq_confirm_code_df)

,code,description
0,1,perfectly validated
1,2,"partially validated, >= 50% coverage"
2,3,wild type
3,4,"partial wild type, >= 50% coverage"
4,5,"target and off-target mutation, >= 50% coverage"
5,6,"off-target mutation, >= 50% coverage"
6,7,"truncated, < 50% coverage"
7,99,no reads


In [18]:
seq_confirm_res_df = pd.read_table(f"{allele_collection_inputs}/VarChampSeqConfirmationResult.tsv")
with pd.option_context('display.max_columns', None):
    display(seq_confirm_res_df)
seq_confirm_res_df["dualip_plate"].unique()

,symbol,ensembl_gene_id,orf_id_wt,mutation_id_old,ccsb_mutation_id,ccsb_allele_id,spdi,nt_change,aa_change,collection,entry_plate_orig,entry_well_orig,entry_plate_conso,entry_well_conso,entry_seq_pool,db_plate,db_well,n2h_plate,n2h_well,dualip_plate,dualip_well,mislocalization_plate,mislocalization_well,entry_sequenced,entry_sequence_confirmation_class,db_sequenced,db_sequence_confirmation_class,n2h_sequenced,n2h_sequence_confirmation_class,dualip_sequenced,dualip_sequence_confirmation_class,mislocalization_sequenced,mislocalization_sequence_confirmation_class
0,GBA1,ENSG00000177628,2,6,CCSBVarC000001,ALE0000584,NC_000001.11:155240033:C:G,160G>C,Val54Leu,RC4,RC4_Mut_GDEh1026,H01,GDEhDisVCh_40054,F12,2.0,RC4_Mut_GDDh1026,H01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,2.0,0,NaN,0,NaN,0,NaN
1,GBA1,ENSG00000177628,2,73,CCSBVarC000002,ALE00000002,NC_000001.11:155238225:G:A,670C>T,Leu224Phe,RC4,RC4_Mut_GDEh1026,E01,GDEhDisVCh_40054,C12,2.0,RC4_Mut_GDDh1026,E01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,1.0,0,NaN,0,NaN,0,NaN
2,GBA1,ENSG00000177628,2,113,CCSBVarC000003,ALE00000003,NC_000001.11:155237453:C:T,887G>A,Arg296Gln,RC4,RC4_Mut_GDEh1026,F01,GDEhDisVCh_40054,D12,2.0,RC4_Mut_GDDh1026,F01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,7.0,0,NaN,0,NaN,0,NaN
3,GBA1,ENSG00000177628,2,231,CCSBVarC000004,ALE00000004,NC_000001.11:155235252:A:G,1448T>C,Leu483Pro,RC4,RC4_Mut_GDEh1026,G01,GDEhDisVCh_40054,E12,2.0,RC4_Mut_GDDh1026,G01,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,2.0,0,NaN,0,NaN,0,NaN
4,GBA1,ENSG00000177628,2,213510,CCSBVarC003869,ALE00003869,NC_000001.11:155239934:G:A,259C>T,Arg87Trp,CEGS2,CegsMutGDEh1035,B03,GDEhDisVCh_40054,B02,2.0,CegsMutGDDh1035,B03,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,2.0,0,NaN,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8705,CTNNB1,ENSG00000168036,100070227,212487,CCSBVarC007073,ALE00007071,NC_000003.12:41225785:A:G,860A>G,Asn287Ser,CEGS2,CegsMutGDEh1023,B07,GDEhDisVCh_40034,E09,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,NaN,0,NaN,0,NaN,0,NaN
8706,TBX19,ENSG00000143178,100070258,212488,CCSBVarC007074,ALE00007072,NC_000001.11:168291339:C:T,383C>T,Ser128Phe,CEGS2,CegsMutGDEh1043,A07,GDEhDisVCh_40046,B02,2.0,CegsMutGDDh1043,A07,NaN,NaN,NaN,NaN,NaN,NaN,1,7.0,1,7.0,0,NaN,0,NaN,0,NaN
8707,TBX19,ENSG00000143178,100070258,212489,CCSBVarC007075,ALE00007073,NC_000001.11:168291213:T:G,257T>G,Met86Arg,CEGS2,CegsMutGDEh1043,G08,GDEhDisVCh_40046,F02,2.0,CegsMutGDDh1043,G08,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,1,1.0,0,NaN,0,NaN,0,NaN
8708,GDF2,ENSG00000263761,100070273,212491,CCSBVarC007076,ALE00007074,NC_000010.11:47322871:G:T,203G>T,Arg68Leu,CEGS2,CegsMutGDEh1023,E04,GDEhDisVCh_40033,B08,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5.0,0,NaN,0,NaN,0,NaN,0,NaN


array([nan, 'VUSMutpDEST2_01', 'VUSMutpDEST2_02', 'VUSMutpDEST2_03',
       'VUSMutpDEST2_04', 'VUSMutpDEST2_05', 'VUSMutpDEST2_06',
       'VUSMutpDEST2_07', 'VUSMutpDEST2_08', 'VUSMutpDEST2_09',
       'VUSMutpDEST2_11', 'VUSMutpDEST2_12', 'VUSMutpDEST2_13',
       'VUSMutpDEST2_14', 'VUSMutpDEST2_15', 'VUSMutpDEST2_16'],
      dtype=object)

In [19]:
pdest_layout_seq_conf_df = pd.merge(pdest_layout_df, 
        seq_confirm_res_df[
        ["symbol","orf_id_wt","mutation_id_old","spdi","dualip_sequenced","dualip_sequence_confirmation_class","aa_change","nt_change","ccsb_mutation_id"]
        ].rename({
            "orf_id_wt": "orf_id",
            "mutation_id_old": "mut_id"
        }, axis=1), 
    on=["orf_id","mut_id"],
    how="left"
)
print(pdest_layout_seq_conf_df["dualip_sequence_confirmation_class"].unique())

## QC with allele collection df for consistency
meta_cols = ['orf_id', 'mut_id', 'aa_change', 'nt_change', 'symbol']
pdest_layout_seq_conf_df = pdest_layout_seq_conf_df.merge(allele_info_df[meta_cols], on=['orf_id', 'mut_id'], how='left', suffixes=("", "_ac"))
pdest_layout_seq_conf_df

assert(all(pdest_layout_seq_conf_df.dropna(subset="symbol")["symbol"]==pdest_layout_seq_conf_df.dropna(subset="symbol")["symbol_ac"]))
assert(all(pdest_layout_seq_conf_df.dropna(subset="aa_change")["aa_change"]==pdest_layout_seq_conf_df.dropna(subset="aa_change")["aa_change_ac"]))
assert(all(pdest_layout_seq_conf_df.dropna(subset="nt_change")["nt_change"]==pdest_layout_seq_conf_df.dropna(subset="nt_change")["nt_change_ac"]))
pdest_layout_seq_conf_df = pdest_layout_seq_conf_df.drop(columns=["symbol_ac","aa_change_ac","nt_change_ac"])

## replace NAs
pdest_layout_seq_conf_df["orf_id"] = pdest_layout_seq_conf_df["orf_id"].astype("Int64").astype(str)
pdest_layout_seq_conf_df["orf_id"] = pdest_layout_seq_conf_df["orf_id"].replace("<NA>", np.nan)
pdest_layout_seq_conf_df

## Replace missing symbols
pdest_layout_seq_conf_df[["aa_change", "nt_change"]] = pdest_layout_seq_conf_df[["aa_change", "nt_change"]].fillna("WT")
pdest_layout_seq_conf_df["symbol"] = pdest_layout_seq_conf_df["symbol"].fillna(pdest_layout_seq_conf_df["orf_id"].map(orf_to_symbol))

[nan  1.  5.  6.  4.  3.  7. 99.  2.]


In [20]:
pdest_layout_seq_conf_df.to_csv(f'{dualipa_outputs}/dualipa_experimental_layout.csv', index=None)